#### **Según los resultados del análisis realizado durante el EDA, se procede a llevar a cabo la fase de Extracción, Transformación y Carga (ETL)**

**Observación:** La etapa de extracción ya se realizó en en archivo [StockMaxPredict_WebScraping_LSTM](https://github.com/JersonGB22/DataScience_IBM_StockPredictionLSTM_Project/blob/main/StockMaxPredict_WebScraping_LSTM.ipynb)

In [9]:
# Importación de librerías
import pandas as pd

In [14]:
# Carga del set de datos
df=pd.read_csv("https://raw.githubusercontent.com/JersonGB22/DataScience_IBM_StockPredictionLSTM_Project/main/DataStorage/datasets_company.csv")
df["Date"]=pd.to_datetime(df.Date)
df

,Date,Open,High,Low,Close,Adj Close,Volume,id_company
0,2013-01-02,19.779285,19.821428,19.343929,19.608213,16.791185,560518000,AAPL
1,2013-01-02,12.804000,12.905000,12.663000,12.865500,12.865500,65420000,AMZN
2,2013-01-02,27.440001,28.180000,27.420000,28.000000,28.000000,69846400,META
3,2013-01-02,2.333333,2.363333,2.314000,2.357333,2.357333,17922000,TSLA
4,2013-01-02,27.250000,27.730000,27.150000,27.620001,22.620338,52899300,MSFT
...,...,...,...,...,...,...,...,...
18867,2023-09-18,137.630005,139.929993,137.630005,138.960007,138.960007,16233600,GOOG
18868,2023-09-18,140.479996,141.750000,139.220001,139.979996,139.979996,42823500,AMZN
18869,2023-09-18,145.770004,146.479996,145.059998,145.089996,145.089996,2508100,IBM
18870,2023-09-18,298.190002,303.600006,297.799988,302.549988,302.549988,14234200,META


In [15]:
# Información del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18872 entries, 0 to 18871
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        18872 non-null  datetime64[ns]
 1   Open        18872 non-null  float64       
 2   High        18872 non-null  float64       
 3   Low         18872 non-null  float64       
 4   Close       18872 non-null  float64       
 5   Adj Close   18872 non-null  float64       
 6   Volume      18872 non-null  int64         
 7   id_company  18872 non-null  object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 1.2+ MB


Creación de una función para el ajuste de la periocidad a un día (24h) del dataset para cada compañía. Primero se transforma el DataFrame en una serie temporal con índice de fecha y se ajusta su frecuencia a un día ("D"). Luego, se utiliza `bfill` para rellenar los valores faltantes con el valor del día siguiente. Finalmente, restablece el índice para obtener un DataFrame resultante con valores diarios completos.

In [18]:
def periodicity_adjustment(dataframe):
  df=dataframe.copy()
  df_list=[]
  for c in df.id_company.unique():
    df2=df[df.id_company==c]
    df2=df2.set_index("Date").asfreq(freq="D", method="bfill").reset_index()
    df_list.append(df2)
  df_final=pd.concat(df_list).sort_values(by="Date")
  return df_final

In [19]:
df2=periodicity_adjustment(df)
df2

,Date,Open,High,Low,Close,Adj Close,Volume,id_company
0,2013-01-02,19.779285,19.821428,19.343929,19.608213,16.791185,560518000,AAPL
0,2013-01-02,27.250000,27.730000,27.150000,27.620001,22.620338,52899300,MSFT
0,2013-01-02,2.333333,2.363333,2.314000,2.357333,2.357333,17922000,TSLA
0,2013-01-02,12.804000,12.905000,12.663000,12.865500,12.865500,65420000,AMZN
0,2013-01-02,17.918339,18.107130,17.846855,18.013729,18.013729,102033017,GOOG
...,...,...,...,...,...,...,...,...
3911,2023-09-18,298.190002,303.600006,297.799988,302.549988,302.549988,14234200,META
3911,2023-09-18,327.799988,330.399994,326.359985,329.059998,329.059998,16834200,MSFT
3911,2023-09-18,140.479996,141.750000,139.220001,139.979996,139.979996,42823500,AMZN
3911,2023-09-18,271.160004,271.440002,263.760010,265.279999,265.279999,101543300,TSLA


In [21]:
for c in df2.id_company.unique():
  print(f"{c}: ")
  mask=df2[df2.id_company==c].Date.diff().dt.total_seconds()/3600
  print(mask.value_counts(),"\n")

AAPL: 
24.0    3911
Name: Date, dtype: int64 

MSFT: 
24.0    3911
Name: Date, dtype: int64 

TSLA: 
24.0    3911
Name: Date, dtype: int64 

AMZN: 
24.0    3911
Name: Date, dtype: int64 

GOOG: 
24.0    3911
Name: Date, dtype: int64 

IBM: 
24.0    3911
Name: Date, dtype: int64 

META: 
24.0    3911
Name: Date, dtype: int64 



In [23]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27384 entries, 0 to 3911
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        27384 non-null  datetime64[ns]
 1   Open        27384 non-null  float64       
 2   High        27384 non-null  float64       
 3   Low         27384 non-null  float64       
 4   Close       27384 non-null  float64       
 5   Adj Close   27384 non-null  float64       
 6   Volume      27384 non-null  int64         
 7   id_company  27384 non-null  object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 1.9+ MB


Se observa que la periodicidad ahora en todas las compañías es de 24h, además de que se incrementó el número de registros a 27384 y no existe ningún valor nulo. Sin embargo, es necesario verificar el promedio de cada variable para asegurarnos de que los valores faltantes se han llenado adecuadamente utilizando el método `bfill`.

In [26]:
# Verificación del promedio de cada columna en cada compañía
for c in df2.id_company.unique():
  mask_df=df[df.id_company==c]
  mask_df2=df2[df2.id_company==c]
  print(f"Compañía {c}:")
  for i in mask_df2.columns[1:-1]:
    print(f"Promedios original/modificado {i}: {round(mask_df[i].mean(),2)}/{round(mask_df2[i].mean(),2)}")
  print("-----------------------------------------------------------------\n")

Compañía AAPL:
Promedios original/modificado Open: 69.35/69.36
Promedios original/modificado High: 70.12/70.16
Promedios original/modificado Low: 68.62/68.66
Promedios original/modificado Close: 69.4/69.45
Promedios original/modificado Adj Close: 67.48/67.53
Promedios original/modificado Volume: 164103390.54/162836109.79
-----------------------------------------------------------------

Compañía MSFT:
Promedios original/modificado Open: 133.46/133.52
Promedios original/modificado High: 134.81/134.86
Promedios original/modificado Low: 132.09/132.17
Promedios original/modificado Close: 133.51/133.59
Promedios original/modificado Adj Close: 128.5/128.58
Promedios original/modificado Volume: 32008132.46/31405008.15
-----------------------------------------------------------------

Compañía TSLA:
Promedios original/modificado Open: 82.07/82.07
Promedios original/modificado High: 83.91/84.02
Promedios original/modificado Low: 80.1/80.14
Promedios original/modificado Close: 82.05/82.18
Promed

Se observa que los promedios son prácticamente idénticos en todas las variables de todas las compañías, con la excepción de la variable `Volume`. Sin embargo, dado que esta variable no se utilizará en el modelo de Deep Learning, según el EDA, su diferencia no tendrá un impacto significativo.

In [27]:
# Exportación del dataset tratado
df2.to_csv("datasets_company_etl.csv", index=False)

**Conclusión:** Según el EDA, dado que el único problema identificado en nuestro conjunto de datos fue la periodicidad, el proceso de ETL concluye de esta manera. El archivo de datos tratado en formato CSV se utilizará en las etapas relacionadas con el modelo de Deep Learning. Además, se creará una base de datos en MySQL que contendrá tablas individuales para cada una de las compañías, así como tablas de calendario y de compañías con el conjunto de datos tratado, con el propósito de habilitar la creación de dashboards en Power BI.